In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import decomposition
import altair as alt

Read in copy number variant (CNV) and population information data

In [6]:
# read in cnv data
df = pd.read_csv('hm3_cnv_submission.txt', sep = '\t')

df.head()

,cnp_id,chr,start,end,NA06984,NA06985,NA06986,NA06989,NA06991,NA06993,...,NA21733,NA21738,NA21739,NA21740,NA21741,NA21768,NA21776,NA21784,NA21825,NA21826
0,HM3_CNP_1,1,8105049,8112441,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,HM3_CNP_2,1,10292133,10300570,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,HM3_CNP_3,1,10466423,10467633,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,HM3_CNP_4,1,12764515,12894420,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0
4,HM3_CNP_5,1,13647613,13649415,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0


In [7]:
# read in sample information into lists

YRI_file = open('YRI_samples.txt', 'r')
JPT_file = open('JPT_samples.txt', 'r')
CEU_file = open('CEU_samples.txt', 'r')
CHB_file = open('CHB_samples.txt', 'r')

YRI_data = YRI_file.read()
YRI = YRI_data.split('\n')

JPT_data = JPT_file.read()
JPT = JPT_data.split('\n')

CEU_data = CEU_file.read()
CEU = CEU_data.split('\n')

CHB_data = CHB_file.read()
CHB = CHB_data.split('\n')

YRI_file.close()
JPT_file.close()
CEU_file.close()
CHB_file.close()

In [12]:
# remove empty string at end of line
YRI.remove('')
JPT.remove('')
CEU.remove('')
CHB.remove('')

['NA18500',
 'NA18502',
 'NA18501',
 'NA18503',
 'NA18505',
 'NA18504',
 'NA18506',
 'NA18508',
 'NA18507',
 'NA18860',
 'NA18858',
 'NA18859',
 'NA18515',
 'NA18517',
 'NA18516',
 'NA18521',
 'NA18523',
 'NA18522',
 'NA18872',
 'NA18870',
 'NA18871',
 'NA18854',
 'NA18852',
 'NA18853',
 'NA18857',
 'NA18855',
 'NA18856',
 'NA18863',
 'NA18861',
 'NA18862',
 'NA18914',
 'NA18912',
 'NA18913',
 'NA19094',
 'NA19093',
 'NA19092',
 'NA19103',
 'NA19102',
 'NA19101',
 'NA19139',
 'NA19137',
 'NA19138',
 'NA19202',
 'NA19201',
 'NA19200',
 'NA19173',
 'NA19172',
 'NA19171',
 'NA19205',
 'NA19204',
 'NA19203',
 'NA19211',
 'NA19209',
 'NA19210',
 'NA19208',
 'NA19206',
 'NA19207',
 'NA19161',
 'NA19159',
 'NA19160',
 'NA19221',
 'NA19222',
 'NA19223',
 'NA19120',
 'NA19116',
 'NA19119',
 'NA19142',
 'NA19140',
 'NA19141',
 'NA19154',
 'NA19152',
 'NA19153',
 'NA19145',
 'NA19143',
 'NA19144',
 'NA19129',
 'NA19127',
 'NA19128',
 'NA19132',
 'NA19131',
 'NA19130',
 'NA19100',
 'NA19099',
 'NA